<a href="https://colab.research.google.com/github/elhamsh93/handwriting-text-recognition/blob/main/OrigamiNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

/bin/bash: nvidia-smi: command not found


# Download Dataset

In [3]:
%%shell
wget --content-disposition  https://zenodo.org/record/439811/files/Train-B_batch{1,2}.tbz2?download=1

--2023-05-21 17:27:40--  https://zenodo.org/record/439811/files/Train-B_batch1.tbz2?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1889479353 (1.8G) [application/octet-stream]
Saving to: ‘Train-B_batch1.tbz2’

Train-B_batch1.tbz2 100%[===================>]   1.76G  18.1MB/s    in 1m 44s  

2023-05-21 17:29:26 (17.3 MB/s) - ‘Train-B_batch1.tbz2’ saved [1889479353/1889479353]

--2023-05-21 17:29:26--  https://zenodo.org/record/439811/files/Train-B_batch2.tbz2?download=1
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1897592301 (1.8G) [application/octet-stream]
Saving to: ‘Train-B_batch2.tbz2’

Train-B_batch2.tbz2 100%[===================>]   1.77G  18.5MB/s    in 1m 43s  

2023-05-21 17:31:09 (17.6 MB/s) - ‘Train-B_batch2.tbz2’ saved [1897592301/1897592301]

FINISHED --2023-05-21 17:31:09--
To

In [4]:
%%shell

mkdir -p ich17

cat Train-B_batch{1,2}.tbz2 | tar -jxvf - -i --strip-components=2 -C ich17

Streaming output truncated to the last 5000 lines.
Train-B/batch2/page/017380.xml
Train-B/batch2/page/018660.xml
Train-B/batch2/page/016562.xml
Train-B/batch2/page/019454.xml
Train-B/batch2/page/017959.xml
Train-B/batch2/page/015159.xml
Train-B/batch2/page/018948.xml
Train-B/batch2/page/018836.xml
Train-B/batch2/page/017461.xml
Train-B/batch2/page/017039.xml
Train-B/batch2/page/015579.xml
Train-B/batch2/page/017233.xml
Train-B/batch2/page/017338.xml
Train-B/batch2/page/015519.xml
Train-B/batch2/page/015387.xml
Train-B/batch2/page/016760.xml
Train-B/batch2/page/016099.xml
Train-B/batch2/page/018260.xml
Train-B/batch2/page/015459.xml
Train-B/batch2/page/018670.xml
Train-B/batch2/page/016737.xml
Train-B/batch2/page/017661.xml
Train-B/batch2/page/016464.xml
Train-B/batch2/page/016027.xml
Train-B/batch2/page/017578.xml
Train-B/batch2/page/016326.xml
Train-B/batch2/page/019332.xml
Train-B/batch2/page/018132.xml
Train-B/batch2/page/019729.xml
Train-B/batch2/page/019716.xml
Train-B/batch2/page

In [5]:
!ls

ich17  sample_data  Train-B_batch1.tbz2  Train-B_batch2.tbz2


In [6]:
import xml.etree.ElementTree as ET
from xml.sax.saxutils import unescape
import glob
import sys,os
import scipy.misc
from PIL import Image
import os.path as osp
import unicodedata

base  = '/content/ich17'

xml    = osp.join(base,'page/')
pages  = glob.glob(xml+'*.xml')

rm = ["§","æ","­","|","‰","#","+","[","]","œ","̃","‒","*","□","°","†","‹","›","ο","—","£","τ","ν","‡","ψ","ι","α","κ","ω","η","℔","	","χ","ξ","₤","ε","π","~","μ","¬","Ζ","λ","Τ","Γ","І","̸","∫","·",">","♂","✓","Œ","♀","$","∆","ø","ρ","∇"]
print(len(pages))
a=[]
for i in range(len(pages)):
    rt = ET.parse(pages[i]).getroot()
    un = rt.find(".//{http://schema.primaresearch.org/PAGE/gts/pagecontent/2013-07-15}Unicode")
    txt = unescape(un.text, {"&apos;": "'", "&quot;": '"'})
    txt = unicodedata.normalize('NFKD', txt)
    txt = txt.translate({ord(i): None for i in rm})
    
    a.append(txt)
    open(osp.join(base,pages[i][-10:-4]+'.txt'),'w+').write(txt.strip())

10000


# Load Dataset

In [7]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%%shell

cp /content/drive/MyDrive/ICDAR/test.txt /content/
cp /content/drive/MyDrive/ICDAR/train.txt /content/

In [9]:
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision
import torchvision.transforms as tr

In [10]:
def get_alphabet(labels):
  unq = sorted(list(set(''.join(labels))))
  alph = dict(zip(unq, range(len(unq))))
  ralph = dict(zip(range(len(unq)), unq))
  return alph, ralph

In [11]:
class ICDARDataset(Dataset):
  def __init__(self, path='/content/ich17', file='/content/train', alph=None, ralph=None):
    #img path
    self.fnames = open(file, 'r').readlines()
    self.fnames = [path + x.strip() for x in self.fnames]
    # txt
    self.labels = []
    for i, img_file in enumerate(self.fnames):
      txt = open(img_file[:-3] + 'txt', 'r').read()
      txt = ' '.join(txt.split())
      self.labels.append(txt)

    # Get alphabet
    if alph == None:
      self.alph, self.ralph = get_alphabet(self.labels)
    else:
      self.alph = alph
      self.ralph = ralph
    self.transform = tr.Compose([
        tr.Resize((700, 500)),
        tr.ToTensor(),
        tr.Normalize(mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225])
    ])

  def __getitem__(self, idx):
    img = Image.open(self.fnames[idx])
    img = self.transform(img)
    txt = self.labels[idx]
    return img, txt

  def __len__(self):
    return len(self.labels)

In [12]:
train_dataset = ICDARDataset(path='/content/ich17/', file='/content/train.txt')
test_dataset = ICDARDataset(path='/content/ich17/', file='/content/test.txt',
                            alph=train_dataset.alph, ralph=train_dataset.ralph)

In [13]:
train_loader = DataLoader(dataset=train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

In [14]:
next(iter(train_loader))

[tensor([[[[2.2318, 2.2318, 2.2318,  ..., 1.6153, 1.6495, 1.8722],
           [2.2147, 2.2147, 2.2318,  ..., 1.0331, 1.0502, 1.2557],
           [1.8722, 1.8550, 1.8722,  ..., 1.0502, 1.1015, 1.3413],
           ...,
           [1.3755, 1.2043, 1.2385,  ..., 2.2489, 2.2489, 2.2489],
           [1.4954, 1.3584, 1.3413,  ..., 2.2489, 2.2489, 2.2489],
           [2.0605, 1.9407, 1.9407,  ..., 2.2489, 2.2489, 2.2489]],
 
          [[2.4111, 2.4111, 2.4111,  ..., 1.7283, 1.7633, 2.0084],
           [2.3936, 2.3936, 2.4111,  ..., 1.1506, 1.1681, 1.3606],
           [2.0434, 2.0259, 2.0259,  ..., 1.1856, 1.2381, 1.4832],
           ...,
           [1.5007, 1.3256, 1.3782,  ..., 2.4286, 2.4286, 2.4286],
           [1.6232, 1.4832, 1.4657,  ..., 2.4286, 2.4286, 2.4286],
           [2.2185, 2.0784, 2.0784,  ..., 2.4286, 2.4286, 2.4286]],
 
          [[2.6226, 2.6226, 2.6400,  ..., 1.7511, 1.7685, 2.0300],
           [2.5703, 2.5703, 2.6051,  ..., 1.1237, 1.1237, 1.3328],
           [2.1171, 2.09

In [15]:
from functools import partial
from typing import Any, Callable, List, Optional, Type, Union

import torch
import torch.nn as nn
from torch import Tensor


__all__ = [
    "ResNet",
    "resnet18",
    "resnet34",
    "resnet50",
    "resnet101",
    "resnet152",
]


def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError("BasicBlock only supports groups=1 and base_width=64")
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition" https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.0)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 1000,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                f"or a 3-element tuple, got {replace_stride_with_dilation}"
            )
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, padding=3, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        # self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], stride=2)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=(1, 2), dilate=replace_stride_with_dilation[2])
        self.layer5 = self._make_layer(block, 512, layers[4])
        # Decoder
        self.interp1 = nn.Upsample(size=(3500, 16), mode='bilinear')
        self.layer6 = self._make_layer(block, 512, layers[5])
        self.interp2 = nn.Upsample(size=(7200, 8), mode='bilinear')
        self.conv7 = nn.Conv2d(512, 92, kernel_size=1)
        
        self.avgpool = nn.AdaptiveAvgPool2d((None, 1))
        self.in2 = nn.InstanceNorm2d(92)
        # self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck) and m.bn3.weight is not None:
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock) and m.bn2.weight is not None:
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

    def _make_layer(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        planes: int,
        blocks: int,
        stride: int = 1,
        dilate: bool = False,
    ) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                )
            )

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        # x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)

        x = self.interp1(x)
        x = self.layer6(x)
        x = self.interp2(x)
        x = self.conv7(x)
        x = self.avgpool(x)
        x = self.in2(x)
        # x = self.avgpool(x)
        # x = torch.flatten(x, 1)
        # x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


def _resnet(
    block: Type[Union[BasicBlock, Bottleneck]],
    layers: List[int],
    progress: bool,
    **kwargs: Any,
) -> ResNet:

    model = ResNet(block, layers, **kwargs)

    return model


In [16]:

def resnet26(*,  progress: bool = True, **kwargs: Any) -> ResNet:
    """ResNet-26 from `Deep Residual Learning for Image Recognition <https://arxiv.org/pdf/1512.03385.pdf>`__.

    Args:
        weights (:class:`~torchvision.models.ResNet34_Weights`, optional): The
            pretrained weights to use. See
            :class:`~torchvision.models.ResNet34_Weights` below for
            more details, and possible values. By default, no pre-trained
            weights are used.
        progress (bool, optional): If True, displays a progress bar of the
            download to stderr. Default is True.
        **kwargs: parameters passed to the ``torchvision.models.resnet.ResNet``
            base class. Please refer to the `source code
            <https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py>`_
            for more details about this class.

    .. autoclass:: torchvision.models.ResNet25_Weights
        :members:
    """
    return _resnet(BasicBlock, [1, 2, 5, 3, 1, 3], progress, **kwargs)

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [18]:
model = resnet26()
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding

In [19]:
model(torch.randn(1, 3, 700, 500).to(device)).shape

torch.Size([1, 92, 7200, 1])

# Config

In [65]:
loss_fn = torch.nn.CTCLoss(reduction='none', zero_infinity=True).to(device)


In [37]:
class CTCLabelConverter(object):
    """ Convert between text-label and text-index """

    def __init__(self, character):
        # character (str): set of the possible characters.
        dict_character = list(character)

        self.dict = {}
        for i, char in enumerate(dict_character):
            # NOTE: 0 is reserved for 'blank' token required by CTCLoss
            self.dict[char] = i + 1

        self.character = ['[blank]'] + dict_character  # dummy '[blank]' token for CTCLoss (index 0)

    def encode(self, text):
        """convert text-label into text-index.
        input:
            text: text labels of each image. [batch_size]

        output:
            text: concatenated text index for CTCLoss.
                    [sum(text_lengths)] = [text_index_0 + text_index_1 + ... + text_index_(n - 1)]
            length: length of each text. [batch_size]
        """
        length = [len(s) for s in text]
        text = ''.join(text)
        text = [self.dict[char] for char in text]

        return (torch.IntTensor(text).to(device), torch.IntTensor(length).to(device))

    def decode(self, text_index, length):
        """ convert text-index into text-label. """
        texts = []
        index = 0
        for l in length:
            t = text_index[index:index + l]

            char_list = []
            for i in range(l):
                if t[i] != 0 and (not (i > 0 and t[i - 1] == t[i])) and t[i]<len(self.character):  # removing repeated characters and blank.
                    char_list.append(self.character[t[i]])
            text = ''.join(char_list)

            texts.append(text)
            index += l
        return texts

In [38]:
train_dataset[100][0].shape, train_dataset[100][0].unsqueeze(0).shape

(torch.Size([3, 700, 500]), torch.Size([1, 3, 700, 500]))

In [39]:
train_dataset[100][1], len(train_dataset[100][1])

('Betriebdirektors der Norditalienischen Bahnen in Uebereinstimmung steht. Derselbe äußerte sich nehmlich vorgestern gegen einen Ingenieur von Como, «man möge sich lange für den Gotthardt bemühen, es werde doch Nichts helfen, denn am Ende würden Andere siegen.» Du weißt, daß ich Dir schon wieder- holt bemerkt habe, die Haltung der Pariser Finanzmächte und ihr Einfluss in Italien beunruhige mich, ich kann nicht umhin, mich auch heute in gleichem Sinne auszu- sprechen. Doch der Postabgang drängt, da ich das franz. Exemplar der Konzession noch gebrauche, so lege ich das deutsche bei. Du wirst Dich in Betreff der Abänderungen',
 635)

In [40]:
image_tensors, labels = next(iter(train_loader))

In [41]:
labels

('Prüfung u mit größter Gewissen- haftigkeit zu antworten, indem ich zugleich mich für versichert halte, daß dergleichen Perso- nalnotizen als durchaus vertraulich behandelt u ange- sehen werden. Mit Vergnügen ergreife ich den Anlass zur Versicherung der ausgezeichnetsten Hochachtg. u Verehrung Ihr ganz ergebener Th Weishaupt. Sofern die Durchführung unserer umfangreichen organisatorischen Einrichtungen u die Einleitungen zum Ausbau zahlreicher Eisenbahn- linien auf Staatskosten, wozu etwa 200 Millionen Thaler von der LandesVertretung bewilligt sind, es gestatten, darf ich hoffen, Sie dieses Jahr dort begrüßen zu dürfen. T Weishpt',)

In [66]:
converter = CTCLabelConverter(train_dataset.ralph.values())

In [67]:
text, length = converter.encode(labels)

In [58]:
length

tensor([643], dtype=torch.int32)

In [59]:
image = image_tensors.to(device)
batch_size = image.size(0)

In [60]:
batch_size

1

In [61]:
preds = model(image)

In [62]:
preds.shape, preds.squeeze(-1).shape

(torch.Size([1, 92, 7200, 1]), torch.Size([1, 92, 7200]))

In [63]:
pred_size = torch.IntTensor([preds.size(1)] * batch_size).to(device)
preds = preds.squeeze(-1).permute(2, 0, 1).log_softmax(2)

In [68]:
loss_fn(preds, text.to(device), pred_size, length.to(device))

tensor([0.], grad_fn=<WhereBackward0>)

In [69]:
lr = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=10**(-1/90000))

In [ ]:
for image, targets in train_loader:
  images = image.to(device)
  text, length = converter.encode(targets)

  preds = model(images).squeeze(-1)
  pred_size = torch.IntTensor([preds.size(1)] * batch_size).to(device)
  preds = preds.permute(2, 0, 1).log_softmax(2)
  cost = loss_fn(preds, text.to(device), pred_size, length.to(device)).mean()
  print(cost)
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if lr_scheduler is not None:
    lr_scheduler.step()

tensor(0., grad_fn=<MeanBackward0>)
tensor(0., grad_fn=<MeanBackward0>)
